This task is for Football players' Classification & Regression in FIFA-19 dataset

In this task, you're required to do the data insight and prepare the dataset for model training

Feature engineering is welcomed, you're encourged to extract features as many as you can

You can choose different models to do the classification and regression work, or try to use other tricks to win higher score. (ensemble learning ...)

Have fun! Goal!Goal!Goal!

In [ ]:
import sys
import pandas as pd
import warnings
import re
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
# load data
df_origin = pd.read_csv('./data/Xgboost-origin_data.csv')

In [ ]:
# data distribution
df_origin.describe()

In [ ]:
# check specific data
df_origin.head(2)

In [ ]:
# column info and related data type
df_origin.info()

In [ ]:
# catagory feature
df_origin.select_dtypes(include=object).columns

In [ ]:
#  feature
df_origin.select_dtypes(include=object).columns

In [ ]:
df_origin['StandingTackle']

In [ ]:
# deal with feature one by one

In [ ]:
# drop the unnamed column
df_origin.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
# Name
# duplication of name?
df_origin['Name'].describe()

In [ ]:
df_origin[df_origin['Name']=='J. Rodríguez']

In [ ]:
# drop the name
df_origin.drop(columns='Name',inplace=True)

In [ ]:
# ID
# drop the ID
df_origin.drop(columns='ID',inplace=True)

In [ ]:
# Photo & Flag
# drop the Photo & Flag
df_origin.drop(columns=['Photo','Flag'],inplace=True)

In [ ]:
# Nationality
# drop 
df_origin.drop(columns=['Nationality'],inplace=True)

In [ ]:
# Club Logo
# drop
df_origin.drop(columns=['Club Logo'],inplace=True)

In [ ]:
# Real Face 
# drop
df_origin.drop(columns=['Real Face'],inplace=True)

In [ ]:
# Club
# drop
df_origin.drop(columns=['Club'],inplace=True)

In [ ]:
# Value
# change to float type
df_origin['Value'].isnull().sum()

In [ ]:
df_tmp1 = df_origin['Value'].str.extract('€(.*)([M|K|\S])', expand=True)
df_tmp1[0] = df_tmp1[0].apply(pd.to_numeric)
df_tmp1[0] = df_tmp1.apply(lambda k: k[0]*1000 if k[1]=='M' else (k[0] if k[1]=='K' else 0),axis=1)
df_origin['Value'] = df_tmp1[0]

In [ ]:
# Wage
# change to float type
df_origin['Wage'].isnull().sum()

In [ ]:
df_tmp2 = df_origin['Wage'].str.extract('€(.*)([M|K|\S])', expand=True)
df_tmp2[0] = df_tmp2[0].apply(pd.to_numeric)
df_tmp2[0] = df_tmp2.apply(lambda k: k[0]*1000 if k[1]=='M' else (k[0] if k[1]=='K' else 0),axis=1)
df_origin['Wage'] = df_tmp2[0]

In [ ]:
# To see na in features
df_origin.isnull().sum()

In [ ]:
# Preferred Foot
df_origin[df_origin['Preferred Foot'].isna()].head()

In [ ]:
df_origin.drop(index=df_origin[df_origin['Preferred Foot'].isna()].index, inplace=True)

In [ ]:
df_origin['Preferred Foot'] = df_origin['Preferred Foot'].apply(lambda k : 0 if k == 'Left' else 1)

In [ ]:
# Work Rate
df_tmp3 = df_origin['Work Rate'].str.extract('(.*)/(.*)', expand=True)

In [ ]:
df_tmp3[0].unique()

In [ ]:
df_tmp3[1].unique()

In [ ]:
df_tmp3[0] = df_tmp3[0].apply(lambda k: 1 if k=='Medium' else (0 if k=='Low' else 2))
df_tmp3[1] = df_tmp3[1].apply(lambda k: 1 if k==' Medium' else (0 if k==' Low' else 2))
df_tmp3.columns = ['Offence_rate','Deffence_rate']

In [ ]:
df_origin = df_origin.join(df_tmp3)

In [ ]:
df_origin.drop(columns='Work Rate',inplace=True)

In [ ]:
# Body Type
df_origin.drop(columns='Body Type',inplace=True)

In [ ]:
# Position
df_origin.drop(index=df_origin[df_origin['Position'].isna()].index, inplace=True)

In [ ]:
# change to one-hot
df_origin = df_origin.join(pd.get_dummies(df_origin['Position'],prefix='Pos', drop_first=True))

In [ ]:
df_origin.drop(columns='Position',inplace=True)

In [ ]:
# Jersey Number
df_origin['Jersey Number'].isnull().sum()

In [ ]:
# Join the club
df_origin.drop(columns='Joined',inplace=True)

In [ ]:
# Loamed From
# Check for missing values in columns where missing values is more than half of the total number of values.
df_origin.isnull().sum()[df_origin.isnull().sum() >= 10000]

In [ ]:
# drop
df_origin.drop(columns='Loaned From',inplace=True)

In [ ]:
# To see na in features
df_origin.isnull().sum()

In [ ]:
# Contract Valid Until
# drop
df_origin.drop(columns='Contract Valid Until',inplace=True)

In [ ]:
df_origin.head(5).iloc[:,8:]

In [ ]:
# Height
df_tmp5 = pd.DataFrame(df_origin['Height'].str.split('\'').tolist(),
                                   columns = ['h1','h2'],index = df_origin.index)
df_tmp5['h1'] = df_tmp5['h1'].apply(pd.to_numeric)
df_tmp5['h2'] = df_tmp5['h2'].apply(pd.to_numeric)

In [ ]:
df_tmp6 = df_tmp5.apply(lambda k: k[0]*30.4+k[1]*2.54,axis=1).to_frame(name='Height')

In [ ]:
# drop the old height first
df_origin.drop(columns='Height',inplace=True)

In [ ]:
df_origin = df_origin.join(df_tmp6)

In [ ]:
# Weight
df_tmp7 = df_origin['Weight'].str.extract('(.*)lbs', expand=True)
df_tmp7.columns = ['Weight']
# drop the olde height first
df_origin.drop(columns='Weight',inplace=True)
# add new
df_origin = df_origin.join(df_tmp7)

In [ ]:
# drop the old height first
df_origin.drop(columns='Release Clause',inplace=True)

In [ ]:
df_origin.fillna(value=0, inplace=True)

In [ ]:
# deal with LS	ST	RS	LW	LF	CF	RF	RW....
#Function to convert skill rating at each position.
def skillConverter(val):
    if type(val) == str:
        s1 = val[0:2]
        s2 = val[-1]
        val = int(s1) + int(s2)
        return val
    
    else:
        return val

In [ ]:
skill_columns = ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM',
       'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM',
       'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB']
                      
for col in skill_columns:
    df_origin[col] = df_origin[col].apply(skillConverter)

In [ ]:
df_origin.head(5)

In [ ]:
# get the label 
df_label = df_origin['Overall'].to_frame()
# drop the label
df_origin.drop(columns=['Overall'],inplace=True)

In [ ]:
df_origin.index.name = "index"

In [ ]:
df_origin.drop(columns=['Unnamed: 0.1'],inplace=True)

In [ ]:
df_label.index.name = "index"
df_label.columns.values[0] = "label"

In [ ]:
# save the data for model training

In [ ]:
df_label.to_csv('./labels.csv',)
df_origin.to_csv('./features.csv')